Import the libraries

In [2]:
import numpy as np
import pandas as pd 

# For portfolio optimization
from scipy.optimize import minimize 
import plotly.graph_objs as go

### 1. Reading Finance Data

In [3]:
# import the data from  local file
stock_data=pd.read_excel('Daily_Returns.xlsx')

# Convert the 'date' column to datetime format 
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Set the 'date' column as the index
stock_data.set_index('Date', inplace=True)

### 2. IMP Functions for Portfolio Construction

In [5]:
# get the names of the stocks
def get_stock_names(df):
    stock_names = df.columns[0:].tolist()
    return stock_names

names = get_stock_names(stock_data)

In [6]:
# getting covarince matrix for portfolios

def get_cov_matrix(df):
    # Calculate the covariance matrix for each stock
    cov_matrix = df.cov()
    
    return cov_matrix

# calling the function
cov_matrix = get_cov_matrix(stock_data)

In [7]:
# getting ER of the portfolio

def get_expected_returns(df):
    
    # Calculate the mean return for each stock
    expected_returns = df.mean()
    
    return expected_returns

expected_returns = get_expected_returns(stock_data)

In [8]:
#getting the correlation matrix
def get_cor_matrix(df):
    # Calculate the correlation matrix for each stock
    cor_matrix = df.corr()
    
    return cor_matrix

cor_matrix=get_cor_matrix(stock_data)


### 3. Optimally Weighted Portfolios

- To generate optimally weighted portfolis, we use `Mean Variance Optimization`.
- The `SLSQP` algorithm (Sequential Least Squares Programming) allows us to minimize a function with several variables and constraints.
- We are minimizing the negative sharpe ratio s.t. 3 constraints as;
    - Sum of weights should be one
    - Weights should not be negative
    - Weights should be between 0 - 0.5 to allow diversification.

In [11]:
number_of_stocks = len(names)
risk_free_rate = ((1+0.03) ** (1/252)) - 1 #conversion into daily RF

# Assuming you have defined expected_returns, cov_matrix, number_of_stocks, and names earlier

# generatig weights
initial_weights = [1/number_of_stocks] * number_of_stocks

constraints = ({'type': 'eq', 'fun': lambda wt: np.sum(wt) - 1}, # sum of weights to be 1
               {'type': 'ineq', 'fun': lambda wt: wt}, # weight should be positive
               {'type': 'ineq', 'fun': lambda wt: 0.1 - np.sqrt(np.dot(wt, np.dot(cov_matrix, wt.T)))}) # risk constraint

bounds = tuple((0, 0.5) for _ in range(number_of_stocks)) # upper bound is set as 0.5 to facilitate diversification.
algorithm = 'SLSQP'  #Sequential Least Squares Programming

# defining objective to optimise
def objective(wt, expected_returns):
    return -np.sum(wt * expected_returns)  # maximize return

# generating weights
def get_weights(expected_returns, cov_matrix, initial_weights=initial_weights, 
                algorithm=algorithm,bounds=bounds,constraints=constraints):
    
    result = minimize(objective,
                        initial_weights,
                        args=(expected_returns),
                        method=algorithm,
                        bounds=bounds,
                        constraints=constraints)
    
    return result

In [12]:
# calling the function
weights= get_weights(expected_returns,cov_matrix)

In [13]:
# map weights to stock names

def map_weights_to_names(optimized_weights, names):
    weights_dict = dict(zip(names, optimized_weights))
    
    # Filter out stocks with weights greater than 0
    selected_stocks = {stock: weight for stock, weight in weights_dict.items() if weight > 0}
    
    return selected_stocks

# function call
optimized_weights = weights.x

selected_stocks = map_weights_to_names(optimized_weights, names)

In [14]:
# Sort the dictionary by weights
sorted_stocks = {k: v for k, v in sorted(selected_stocks.items(), key=lambda item: item[1], reverse=True)}

# Print only the top 10 stocks
count = 0
for stock, weight in sorted_stocks.items():
    print(stock, ":", weight)
    count += 1
    if count == 10:
        break

CF : 0.5
V : 0.5
GOOG : 1.7763568394002505e-15
ALLE : 4.448422220010367e-17
CB : 4.048715241210912e-17
ADP : 4.01512666322222e-17
ADBE : 3.264593770645907e-17
AFL : 3.005467477800077e-17
AKAM : 2.837527952623305e-17
ALGN : 2.6809408493099126e-17


In [15]:
# Calculate the sum of all weights
total_weight = sum(sorted_stocks.values())

print("Total weight:", total_weight)

Total weight: 1.0000000000000018


In [16]:
# get the list of stocks
stock_names=list(sorted_stocks.keys())
print(len(stock_names))
# get the list of corrosponding weights
stock_weights=list(sorted_stocks.values())
print(len(stock_weights))

# portfolio with selected stocks
df_selected = stock_data[['Date'] + stock_names]

26
26


In [17]:
# Calculate the expected return of the portfolio

ER = get_expected_returns(df_selected)

portfolio_expected_return = np.dot(ER, stock_weights)

# Print the expected return of the portfolio
print("Expected return of the portfolio:", portfolio_expected_return)

Expected return of the portfolio: 0.03698236433407959


In [18]:
# get a weighted portfolio

# Extract the 'Date' column
dates = df_selected['Date']

# Multiply each stock's return by its corresponding weight
weighted_returns = df_selected.drop(columns=['Date']).mul(stock_weights)

# Create a new DataFrame with the weighted returns and the 'Date' column
weighted_df = pd.concat([dates, weighted_returns], axis=1)

In [19]:
# calling the function
cov_matrix_weighted = get_cov_matrix(weighted_df)

# get the SD
stock_weights_array = np.array(stock_weights)
portfolio_stddev = np.sqrt(np.dot(stock_weights_array, np.dot(cov_matrix_weighted, stock_weights_array.T)))

# Print the SD of the portfolio
print("SD of the portfolio:", portfolio_stddev)

SD of the portfolio: 0.027319618546578783


In [20]:
# Classical Approach
print("Sharpe Ratio under Traditioanl Method:" , ((portfolio_expected_return-risk_free_rate) / portfolio_stddev))
# QC Approach
# print("Sharpe Ratio under QC approach:" , ((0.2430-risk_free_rate) / 0.2993))

Sharpe Ratio under Traditioanl Method: 1.3493988050159553


### Efficient Frontier

In [21]:
# mu = get_expected_returns(df_selected)
# S = get_cov_matrix(df_selected)

# n_samples = 10000
# np.random.seed(42)
# w = np.random.dirichlet(np.ones(len(mu)), n_samples)
# rets = w.dot(mu)
# stds = np.sqrt((w.T * (S @ w.T)).sum(axis=0))
# sharpes = rets / stds

In [22]:
# # Create scatter plot
# trace = go.Scatter(
#     x=stds,
#     y=rets,
#     mode='markers',
#     marker=dict(
#         size=8,
#         color=sharpes,  # Color based on Sharpe ratio
#         colorscale='Viridis', 
#         showscale=True
#     ),
#     text=['Sharpe Ratio: {:.2f}'.format(sh) for sh in sharpes],
#     name='combinations'
# )

# # Add marker for specific point
# optimized_return = go.Scatter(
#     x=[portfolio_expected_return],
#     y=[portfolio_stddev],
#     mode='markers',
#     marker=dict(
#         size=10,
#         color='red',  # Marker color
#         symbol='star'  # Marker symbol
#     ),
#     name='optimized')

# # Layout
# layout = go.Layout(
#     title='Portfolio Returns vs. Risk',
#     xaxis=dict(title='Risk'),
#     yaxis=dict(title='Returns'),
#     hovermode='closest'
# )

# fig = go.Figure(data=[trace, optimized_return], layout=layout)

# # Plot figure
# fig.show()